# Introduction

The *EM* algorithm consists of two phases: estimation and maximization, *_EM_* for short. 
1. The first phase consist of estimating some latent variables associated with each point given the unknown paramters: mean and covariance.
2. Having the estimation of the hidden variables (aka latent variables), the second step consists of recomputing the unknown parameters (mean and covariance)

Executing this stepts multiple times, the results will yield some local (it can be global) maxima set of parameters. 

Besides the data, the algorithm needs an intialization point: It can be the initial location and shape of the distribution - most of the time it's random, *but the number of clusters/components should be specified*

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mplib
import seaborn as sns
import scipy.stats as stats
from matplotlib import rc
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import ipywidgets as widgets
import itertools as it
from IPython.core.debugger import set_trace
from matplotlib.colors import LogNorm
import functools as fp

In [22]:
%matplotlib widget

# Setting

In [23]:
sns.set()

# Utils

## 1D algorithm

In [87]:
def em(inits, x, priors=None):
    if priors is None:
        priors = it.cycle([1])
    # expectation step
    # compute the likelihoods
    pdfs = np.r_[[stats.multivariate_normal.pdf(x, mu, std) for (mu, std), prior in zip(inits, priors)]]
    # normalize the likelihoods
    pdfs /= np.sum(pdfs, axis=0)
    # maximization step
    # estimate the means and the standard deviation
    mus = np.sum(x * pdfs, axis=1) / np.sum(pdfs, axis=1)
    stds = np.sqrt(np.sum(((x - mus[:, np.newaxis]) ** 2) * pdfs, axis=1) / np.sum(pdfs, axis=1))
    return [(mu, std) for mu, std in zip(mus, stds)]

## 2D algorithm. maybe nD - haven't tested it :(

In [88]:
def multivariate_em(inits, x, priors=None):
    # expectation step
    # computing the numerators
    pdfs = np.r_[[stats.multivariate_normal.pdf(x, mu, std) for mu, std in inits]]
    # normalizing the likelihoods
    pdfs /= (pdfs.sum(0) + np.finfo(float).eps)
    # maximization step
    # estimating the means
    mus = np.sum(pdfs[..., np.newaxis] * x, axis=1) / np.sum(pdfs, axis=1, keepdims=True)
    def comp_cov(x, mu, pdf):
        part = x - mu
        return np.dot(part.T * pdf, part) / pdf.sum()

    # estimatic the covariance matrix
    return [(mu, comp_cov(x, mu, pdf)) for mu, pdf in zip(mus, pdfs)]

# Helpers

In [90]:
def train_em(params, x, priors, em_func):
    """Lazy training, only when the callera requests it"""
    while True:
        new_params = em_func(params, x, priors)
        yield new_params
        params = new_params
        

In [93]:
def snap_em(em_fn, x, params, priors, no):
    """Snaps/Gather all the intermediate params"""
    return [params] + [param for _, param in zip(
            range(no), 
            iter(train_em(params, x, priors, em_fn))
        )]

In [103]:
class Seq1D:
    """Plot the evolution of the 1D EM algorithm"""
    
    def __init__(self, ax, snaps, space):
        self.ax = ax
        self.space = space
        self.snaps = snaps
        self.lines = [
            self.ax.plot(self.space, stats.norm.pdf(self.space, mu, std))[0]
            for mu, std in self.snaps[0]
        ]
        
            
    def plot(self, i):
        y_max = -1
        for line, (mu, std) in zip(self.lines, self.snaps[i]):
            y = stats.norm.pdf(self.space, mu, std)
            y_max = max(y.max(), y_max)
            line.set_data(self.space, y)
        self.ax.set_ylim(-0.002, y_max + 0.002)
        
        


In [104]:
class Seq2D:
    """Plot the evolution of the 2D EM algorithm"""
    
    def __init__(self, ax, snaps, X, Y):
        self.ax = ax
        self.X, self.Y = np.meshgrid(X, Y)
        self.X_O_Y = np.array([self.X.ravel(), self.Y.ravel()]).T
        self.snaps = snaps
        self.contours = []
        
            
    def plot(self, i):
        self.clear()
        for mu, std in self.snaps[i]:
            Z = stats.multivariate_normal.pdf(self.X_O_Y, mu, std)
            Z = Z.reshape(self.X.shape)
            self.contours.append(
                ax.contour(
                    self.X, 
                    self.Y, 
                    Z, 
                    norm=LogNorm(vmin=0.01, vmax=1.0), 
                    levels=np.logspace(-4, 0, 10), 
                    alpha=0.4
                )
            )
            
    def clear(self):
        for contour in self.contours:
            for coll in contour.collections:
                coll.remove()
        self.contours.clear()
        


In [96]:
def compute_space1d(x, how_big=200):
    m_diff = np.mean(np.abs(np.diff(x)))
    return np.linspace(x.min() - m_diff, x.max() + m_diff, how_big)

In [97]:
snap_1dem = fp.partial(snap_em, em)
snap_ndem = fp.partial(snap_em, multivariate_em)

# Example for 1D data

In [102]:
# data sampling
no = 100
blue = np.random.normal(-50, 5, 500)
red = np.random.normal(20, 10, 1000)
ne = np.random.normal(100, 5, 400)
all_ = np.r_[blue, red, ne]
# seting the initial params
blue_mean, blue_std, red_mean, red_std, ne_mean, ne_std = -5, 5, 6.5, 5, 20, 6
inits_1d = [(blue_mean, blue_std), (red_mean, red_std), (ne_mean, ne_std)]
b_p, r_p, n_p = 1 / np.array([len(blue), len(red), len(ne)])

# Ploting
fig, ax = plt.subplots()
ax.scatter(blue, np.zeros_like(blue), marker='x', label='x')
ax.scatter(red, np.zeros_like(red), marker='o', label='o')
ax.scatter(ne, np.zeros_like(ne), marker='o', label='o')
smt = Seq1D(ax, snap_1dem(all_, inits_1d, [b_p, r_p, n_p], no), compute_space1d(all_))
_ = widgets.interact(smt.plot, i=(0, no, 1))


     

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=50, description='i'), Output()), _dom_classes=('widget-interact',))

# Example for 2D data

In [105]:
# data sampling
a = np.random.multivariate_normal([0,0], [[1, 0], [0, 1]], 10000)
b = np.random.multivariate_normal([20,-1], [[-1, 5], [0, 1]], 2000)
c = np.random.multivariate_normal([10,0], [[1, 0], [0, 1]], 3000)

x = np.r_[a,b,c]

# setting the initial params
m0, s0 = [-10,0], [[1, 0], [0, 1]]
m1, s1 = [10,10], [[1, 0], [0, 1]]
m2, s2 = [30,1], [[5, 2], [0, 1]]

inits = [(m0, s0), (m1, s1), (m2, s2)]

# snapshoting the execution of the algorithm
snap_dd = snap_ndem(x, inits, [1/3, 1/3, 1/3], 100)

/home/hsc/anaconda3/envs/dizi/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: covariance is not positive-semidefinite.
  This is separate from the ipykernel package so we can avoid doing imports until


In [106]:
# Ploting
no = 25
fig, ax = plt.subplots()
for v in [a, b, c]:
    ax.scatter(*v.T)
smt = Seq2D(
    ax, 
    snap_ndem(
        x, 
        inits, 
        [1/3, 1/3, 1/3], 
        no
    ), 
    np.linspace(-10., 30.), 
    np.linspace(-10., 10.)
)
_ = widgets.interact(smt.plot, i=(0, no, 1))

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=12, description='i', max=25), Output()), _dom_classes=('widget-interact'…